# MDPにおける安全な探索

参考
* [Safe Exploration in Markov Decision Processes](https://arxiv.org/abs/1205.4810)：正直表記が微妙で分かりづらい．

「安全な探索を実現する方策」とはなんでしょうか？
ある状態$s$を考えてみます．$s$を出発したエージェントが，いずれ再び$s$に戻れるような探索を実現している場合は，この探索は安全であると言えそうです．
つまり，「元の場所に帰ってこれる」ような方策による探索は安全であると言えます （例：追突事故を起こした車は事故前に戻ることはできません）．
今回は，このような安全性をエルゴード性を使って数学的に定義していきます．

表記：
* 総報酬：$V:=\sum_{t=0}^{\infty} r_{S_t, A_t}$.
* ベルマン方程式：$E_{s, \pi}^p[V]=\sum_{a, s^{\prime}} \pi_{s, a}\left(r_{s, a}+p_{s, a, s^{\prime}} E_{s^{\prime}, \pi}^p[V]\right)$
    * $\sum_{s^{\prime}} p_{s, a, s^{\prime}} \leq 1$として，余った値を$\gamma$に割り振ることでInfinite horizonに対応することにします．
* 今回は遷移カーネル$P$が確率変数であるとして，$P$は確率測度$\beta$に従うとします．この確率測度をbeliefと呼びます．
    * 不確実集合全体での期待報酬和を$E_\beta E_{s, \pi}^P[V]$とします．
    * $p:=E_\beta[P]$とします．

---

**エルゴード性**

今回は，あるbelief $\beta$ は次を満たすときにエルゴード性があると言います．

全ての$\beta(P) >0$である$P$について，全ての状態が他の全ての状態から，何らかの方策によって到達可能である．言い換えると，

$$
\forall s, s^{\prime}, \exists \pi_r \text{ such that } E_\beta E_{s, \pi_r}^P\left[B_{s^{\prime}}\right]=1
$$

が成立する．ここで，$B_{s'}$は$s'$を少なくとも一回訪れるイベントとした．つまり，

$$
B_{s'}=1\left\{\exists t<\infty\right. \text{ such that }\; \left.S_t=s^{\prime}\right\}=\min \left(1, \sum_t 1_{S_t=s^{\prime}}\right)
$$

---


## 安全な方策

このエルゴード性を利用して，安全な方策を改めて定義します．

---

**安全な方策**

次を満たすとき，方策$\pi_o$は初期状態$s_0$と停止時刻$T$について$\delta$-safeであると言います．

$$
\exists \pi_r \text { such that } E_\beta E_{s_0, \pi_o}^P\left[E_{S_T, \pi_r}^P\left[B_{s_0}\right]\right] \geq \delta
$$

---

これは例えばロボットヘリコプターの例を考えるとわかりやすいです．
例えば，ヘリコプターが学習を終了した際に，確率$\delta$で安全に着陸できるような場合，このヘリコプターは$\delta$-safeな方策で学習をしていたことになります．
この例では，停止時刻$T$はヘリコプターに着陸命令が出された時刻です．

例えば$\delta=1$のときを考えてみましょう．このとき，期待値の中身は全部１になります．式を内側から見ていくと， 

* $E_{S_T, \pi_r}^P\left[B_{s_0}\right]=1$は遷移確率$P$，初期状態$S_T$で，方策$\pi_r$に従ったとき，必ず$s_0$に帰還できることを意味します．
* $E_{s_0, \pi_o}^P\left[E_{S_T, \pi_r}^P\left[B_{s_0}\right]\right]=1$は初期状態$s_0$から$\pi_o$で動き回り，時刻$T$で訪れる停止状態$S_T$を初期状態にしたときに，$s_0$に帰還できることを意味します．
* $E_\beta$で全ての$\beta(P) >0$な$P$について帰還を保証します．


## 安全になれない簡単な条件について

上の安全な方策の定義は，一見すると冗長な気がします．
例えば安全な状態と行動の集合だけを方策に強制するのは不十分でしょうか？
次の図に示すMDPを考えてみます．

![safe](figs/safety.png)

次の２つの決定的な方策を考えます．

* aababab... で行動を選択する方策
* acccccc... で行動を選択する方策

この２つのうち，前者は0.8-safeな方策ですが，後者は0.8-safeではありません．一方で，どちらも状態Bを経由します．このことから，状態Bは安全とも安全でないとも言えません．そのため，どちらに割り振ればよいか微妙な状態や行動が出てきてしまいます．


また，上の安全な方策の定義では，全てのbeliefについて，同じ帰還用の方策が存在することを条件にしています．
これは強すぎる条件な気がしますね．
例えば次のような条件はどうでしょうか．

$$
E_\beta 1\left\{\exists \pi_r: E_{s_0, \pi_o}^P E_{S_T, \pi_r}^P\left[B_{s_0}\right]=1\right\} \geq \delta
$$

そして，下の図の2つのMDPが，どちらも確率0.5でbeliefから出てくることを考えてみます．

![safety](figs/safety2.png)

このとき，状態$S$から$B$に行くのは安全でしょうか？
$B$に行ったあと，どちらのMDPにも帰還用の方策は存在するため，直前の定義ではこのbeliefは安全になってしまいます．
しかし，例えば$L$で車がクラッシュしてしまう状況を考えると，直感的には，Bに進むのは安全ではありません．
よって直前で定義した安全性の条件は微妙です．


## 安全な方策集合を見つけるのはNP困難

上で安全な方策が満たすべき条件を定義しましたが，そのような方策の集合を見つけるのは，一般にはNP困難です．
証明しましょう．

---

次を証明します．

初期状態$s_0$, 停止時刻$T$，belief $\beta$について，一般に，$\delta$-safeな方策集合が存在するかを示すのはNP困難である．


次の図に示すMDPを考えましょう．

* 行動集合：$A=\{a_0, a_1, a_2, a_3\}$．
    * $a_0$は同じ状態に戻る行動とします．
* 状態集合：$\{S, D, X_1, Y_1, Z_1, \dots, X_n, Y_n, Z_n\}$

![safety](figs/safety3.png)

---

**遷移について**

$i\in [n]$について，$X_i$で$a_j$ with $j \in \{1, 2, 3\}$ を取ったときの遷移を次で定めます．

* もし$X_i=\text{ true}$ならば，$a_1 \to X_{i+1}$，$a_2 \to Y_{i+1}$，$a_3 \to Z_{i+1}$
* もし$X_i=\text{ false}$ならば，どれを選んでも$X_i$に遷移

$Y_i$, $Z_i$についても同様とします．

---

**証明**

以下の$k$個のバイナリの変数の集合$U$を考えましょう．
$$
U=\times_{i\in [k]} \{U_i \mid U_i \in \{\text{ true }, \text{ false }\}\}
$$

上のMDPにおいて，各変数$X_i$，$Y_i$，$Z_i$は$U$の中の変数のどれかと一致しているとします．
beliefは$U$に均等に割り当てられている場合を考えましょう．
初期状態$S$と停止時刻$T$について，このbeliefが$p>0$について$p$-safeであることを言いたいとします．
これを言うには，$U$の中に，最低でも一つ次の論理式をTrueにする要素が存在することを判定しなければなりません．

$$
\left(X_1 \vee Y_1 \vee Z_1\right) \wedge \cdots \wedge\left(X_n \vee Y_n \vee Z_n\right)
$$

しかし，これが多項式時間で解ける場合，[3-SAT問題](https://ja.wikipedia.org/wiki/%E5%85%85%E8%B6%B3%E5%8F%AF%E8%83%BD%E6%80%A7%E5%95%8F%E9%A1%8C)が多項式時間で解けることになってしまいます．
よってsafeな方策集合を見つけるのはNP困難です．

[3-SAT問題の解説はここが詳しいかも](http://www.akita-pu.ac.jp/system/elect/ins/kusakari/japanese/teaching/InfoMath/2007/note/9p.pdf)．

**コメント**: これは結局Non-rectangularなUncertainty set上での最適化が難しいこととほぼ同義だと思われる．Rectangularの仮定がある場合は$X_i$, $Y_i$, $Z_i$は全て独立で，このときは多項式時間で見つかりそう．

---



